In [7]:
#BERTopic for the tales in the original text (first 30 tales)
import os
from langdetect import detect
import string
import nltk
from bertopic import BERTopic
from umap import UMAP
from bertopic.representation import KeyBERTInspired
import pandas as pd

In [2]:
!pip3 install langdetect

In [3]:
path = "./Callaway/OCR/Callaway_Text/"
files = os.listdir(path)

english_text = []

# Read the file and extract English sentences
for file in files:
    with open(path+file, 'r') as source:
        lines = source.readlines()
        for line in lines:
            # Remove leading/trailing whitespace and punctuation
            line = line.strip().strip(';').strip('\n').strip("®").strip("@").strip("*").strip("%")
            line = line.replace(" . ", "")
            line = line.replace("+", "")
            line = line.replace(")", "").replace("[","")
            line = line.replace("(", "").replace("]","")
            line = line.replace("_","")
            line = line.replace("|","")

            translation_table = str.maketrans('', '', string.digits)
            line = line.translate(translation_table)

            # Detect the language of the line
            try :
                lang = detect(line)
                # Check if the detected language is English
                if lang == 'en':
                    if not line == "":
                        english_text.append(line)
            except: 
                pass

#%%

In [11]:
text = " ".join(english_text)

In [25]:
ents = pd.read_csv("ent_dict_en.csv", sep="\t", header=None)
ents.columns = ["Name", "Label"]
ents.index = ents["Name"]

In [28]:
ent_dict = ents.to_dict("index")

In [30]:
for key in ent_dict.keys(): 
    text = text.replace(key, "["+ent_dict[key]["Label"]+"]")

In [31]:
sentences = nltk.sent_tokenize(text)

In [32]:
sentences

['The soldiers arrive at the river ; it divides: they enter ; it closes, and overwhelms them.',
 'The army reached the bank of the river.',
 "They shouted and said, “Where did you cross over?” They said, “In this very place, Do you cross over, and come and kill'us” They said, “No indeed !",
 'You did not cross bere, Tell us.” [PER] and his children rejoice.',
 'They rejoiced ; the children of [PER] played ; they rejoiced exceedingly.',
 'He said, “Do you not see then?',
 'They are come to an end, who were coming to kill us.” He said, “Take up your loads, and let us go to our people.” So they took up their burdens, and set out.',
 '[PER] and many others die in the way ; a few reach their [PER] died in the way.',
 'The people now went by themselves.',
 'His brother, who had been with the children, came, and went with them.',
 'Death came, and killed the old men.',
 'The young remained ; they remained with only one man.',
 'And so they journeyed, and at length came to the country of their

In [45]:
umap = UMAP(min_dist=0.0,
            metric='cosine',
            low_memory=False,
            random_state=42) 

In [46]:
topic_model = BERTopic("English", umap_model=umap)
topics, probs = topic_model.fit_transform(sentences)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1107,-1_the_and_she_of,"[the, and, she, of, to, in, it, is, said, he]",[And as regards the wagtail he told us continu...
1,0,115,0_cannibal_cannibals_meat_you,"[cannibal, cannibals, meat, you, the, said, br...","[So the cannibal came to her people., The cann..."
2,1,93,1_of_in_tales_which,"[of, in, tales, which, the, monsters, who, to,...","[‘Thus Langfellow, “ Only a little hour I was ..."
3,2,89,2_house_chief_wife_is,"[house, chief, wife, is, if, of, cattle, the, ...","[The house of the chief wife., that is, he is ..."
4,3,87,3_per_his_to_people,"[per, his, to, people, sea, was, came, daughte...","[The birth of [PER]., [PER] came., [PER] came ..."
...,...,...,...,...,...
67,66,11,66_bag_put_caught_fell,"[bag, put, caught, fell, full, animals, venomo...","[The bag was full., He fell in with-a snake h..."
68,67,11,67_village_brass_large_such,"[village, brass, large, such, property, town, ...",[such things; their property consists of beads...
69,68,11,68_umgongo_puberty_is_young,"[umgongo, puberty, is, young, customs, her, fa...","[All the people go to dig in the royal garden,..."
70,69,11,69_end_mischief_charging_that,"[end, mischief, charging, that, was, titbit, e...","[And that was the end of it., That was the end..."


In [47]:
topic_model.get_document_info(sentences)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,The soldiers arrive at the river ; it divides:...,13,13_river_water_vessel_dipped,"[river, water, vessel, dipped, went, took, ful...",[As soon as they and fetch me some water from ...,river - water - vessel - dipped - went - took ...,0.869228,False
1,The army reached the bank of the river.,13,13_river_water_vessel_dipped,"[river, water, vessel, dipped, went, took, ful...",[As soon as they and fetch me some water from ...,river - water - vessel - dipped - went - took ...,1.000000,False
2,"They shouted and said, “Where did you cross ov...",42,42_spears_hurled_him_did,"[spears, hurled, him, did, spear, stab, they, ...",[refuse to give up the children.” All hurled: ...,spears - hurled - him - did - spear - stab - t...,0.973456,False
3,"You did not cross bere, Tell us.” [PER] and hi...",-1,-1_the_and_she_of,"[the, and, she, of, to, in, it, is, said, he]",[And as regards the wagtail he told us continu...,the - and - she - of - to - in - it - is - sai...,0.000000,False
4,They rejoiced ; the children of [PER] played ;...,3,3_per_his_to_people,"[per, his, to, people, sea, was, came, daughte...","[The birth of [PER]., [PER] came., [PER] came ...",per - his - to - people - sea - was - came - d...,0.517173,False
...,...,...,...,...,...,...,...,...
3360,In another tale the sea divides at the word of...,-1,-1_the_and_she_of,"[the, and, she, of, to, in, it, is, said, he]",[And as regards the wagtail he told us continu...,the - and - she - of - to - in - it - is - sai...,0.000000,False
3361,"And in the Hottentot fables, in like manner, H...",1,1_of_in_tales_which,"[of, in, tales, which, the, monsters, who, to,...","[‘Thus Langfellow, “ Only a little hour I was ...",of - in - tales - which - the - monsters - who...,0.858520,False
3362,‘These facts thow the wide-spread existence of...,1,1_of_in_tales_which,"[of, in, tales, which, the, monsters, who, to,...","[‘Thus Langfellow, “ Only a little hour I was ...",of - in - tales - which - the - monsters - who...,0.979127,False
3363,Dr. Bleck has shown that the Hottentot languag...,35,35_zulu_haired_ring_hair,"[zulu, haired, ring, hair, long, head, natives...",[Many kings come A distant king hears of her b...,zulu - haired - ring - hair - long - head - na...,0.630060,False


In [50]:
fig = topic_model.visualize_topics()
fig.write_html("callaway_topics_core_original_text.html")
fig2 = topic_model.visualize_barchart()
fig2.write_html("callaway_core_barchart_orig.html")

In [49]:
fig2

In [51]:
new_topics = topic_model.reduce_outliers(sentences, topics, strategy="distributions")
topic_model.update_topics(sentences, topics=new_topics)

2024-08-23 16:39:20,833 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [52]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4,-1_umkampengula_emhlope_du_chailhy,"[umkampengula, emhlope, du, chailhy, ubabuze, ...",[And as regards the wagtail he told us continu...
1,0,157,0_cannibal_cannibals_meat_the,"[cannibal, cannibals, meat, the, you, said, si...","[So the cannibal came to her people., The cann..."
2,1,145,1_of_in_tales_which,"[of, in, tales, which, tale, the, as, by, who,...","[‘Thus Langfellow, “ Only a little hour I was ..."
3,2,115,2_house_chief_wife_is,"[house, chief, wife, is, if, of, the, has, cat...","[The house of the chief wife., that is, he is ..."
4,3,113,3_per_was_by_his,"[per, was, by, his, to, came, of, son, the, he]","[The birth of [PER]., [PER] came., [PER] came ..."
...,...,...,...,...,...
67,66,20,66_bag_animals_put_cannibal,"[bag, animals, put, cannibal, bitten, he, lion...","[The bag was full., He fell in with-a snake h..."
68,67,18,67_village_town_entrance_such,"[village, town, entrance, such, brass, body, l...",[such things; their property consists of beads...
69,68,19,68_umgongo_puberty_is_young,"[umgongo, puberty, is, young, her, all, she, c...","[All the people go to dig in the royal garden,..."
70,69,15,69_end_immediately_was_agi,"[end, immediately, was, agi, roasting, chargin...","[And that was the end of it., That was the end..."


In [53]:
topic_model.get_document_info(sentences)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,The soldiers arrive at the river ; it divides:...,13,13_water_vessel_river_she,"[water, vessel, river, she, dipped, went, into...",[As soon as they and fetch me some water from ...,water - vessel - river - she - dipped - went -...,0.869228,False
1,The army reached the bank of the river.,13,13_water_vessel_river_she,"[water, vessel, river, she, dipped, went, into...",[As soon as they and fetch me some water from ...,water - vessel - river - she - dipped - went -...,1.000000,False
2,"They shouted and said, “Where did you cross ov...",42,42_spears_hurled_did_picked,"[spears, hurled, did, picked, they, shouted, h...",[refuse to give up the children.” All hurled: ...,spears - hurled - did - picked - they - shoute...,0.973456,False
3,"You did not cross bere, Tell us.” [PER] and hi...",3,3_per_was_by_his,"[per, was, by, his, to, came, of, son, the, he]","[The birth of [PER]., [PER] came., [PER] came ...",per - was - by - his - to - came - of - son - ...,0.000000,False
4,They rejoiced ; the children of [PER] played ;...,3,3_per_was_by_his,"[per, was, by, his, to, came, of, son, the, he]","[The birth of [PER]., [PER] came., [PER] came ...",per - was - by - his - to - came - of - son - ...,0.517173,False
...,...,...,...,...,...,...,...,...
3360,In another tale the sea divides at the word of...,1,1_of_in_tales_which,"[of, in, tales, which, tale, the, as, by, who,...","[‘Thus Langfellow, “ Only a little hour I was ...",of - in - tales - which - tale - the - as - by...,0.000000,False
3361,"And in the Hottentot fables, in like manner, H...",1,1_of_in_tales_which,"[of, in, tales, which, tale, the, as, by, who,...","[‘Thus Langfellow, “ Only a little hour I was ...",of - in - tales - which - tale - the - as - by...,0.858520,False
3362,‘These facts thow the wide-spread existence of...,1,1_of_in_tales_which,"[of, in, tales, which, tale, the, as, by, who,...","[‘Thus Langfellow, “ Only a little hour I was ...",of - in - tales - which - tale - the - as - by...,0.979127,False
3363,Dr. Bleck has shown that the Hottentot languag...,35,35_natives_haired_ring_long,"[natives, haired, ring, long, hair, zulu, head...",[Many kings come A distant king hears of her b...,natives - haired - ring - long - hair - zulu -...,0.630060,False


In [54]:
fig3 = topic_model.visualize_barchart()
fig3.write_html("callaway_reduce_outliers_barchart_orig.html")

In [56]:
fig4 = topic_model.visualize_topics()
fig4.write_html("callaway_reduce_outliers_topics.html")
fig4

In [57]:
representation_model = KeyBERTInspired()
topic_model = BERTopic("english", representation_model=representation_model)

In [58]:
topics, probs = topic_model.fit_transform(sentences)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1167,-1_mother_she_her_father,"[mother, she, her, father, house, woman, child...","[They said, “ Your love for her is evident, fo..."
1,0,240,0_father_mother_child_birth,"[father, mother, child, birth, pregnant, son, ...","[The news is published, A messenger set out an..."
2,1,99,1_cannibal_cannibals_cannibalism_meat,"[cannibal, cannibals, cannibalism, meat, eaten...","[The cannibal said, “Child of my sister, you a..."
3,2,97,2_kwa_ekaya_ikwantandane_njalo,"[kwa, ekaya, ikwantandane, njalo, nga, inyama,...",[So the women swear in like manner : ““ Bane w...
4,3,90,3_heir_heirs_chief_house,"[heir, heirs, chief, house, wives, eldest, inh...","[The house of the chief wife., that is, he is ..."
...,...,...,...,...,...
59,58,12,58_seeds_seed_little_berries,"[seeds, seed, little, berries, scattered, pour...",[Have you not a vessel full of castor-oil berr...
60,59,11,59_end_ended_and_was,"[end, ended, and, was, so, that, with, tale, t...","[And that was the end of it., That was the end..."
61,60,11,60_she_her_unseen_visible,"[she, her, unseen, visible, dawn, stealthily, ...","[They went stealthily, being afraid, and looke..."
62,61,11,61_beads_brass_bracelets_ornaments,"[beads, brass, bracelets, ornaments, garment, ...","[‘They had put on bracelets, and ornaments for..."


In [60]:
topic_model.get_document_info(sentences)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,The soldiers arrive at the river ; it divides:...,6,6_river_rivers_water_drowned,"[river, rivers, water, drowned, vessel, vessel...","[the river., On their arrival they said, “Take...",river - rivers - water - drowned - vessel - ve...,0.932876,False
1,The army reached the bank of the river.,6,6_river_rivers_water_drowned,"[river, rivers, water, drowned, vessel, vessel...","[the river., On their arrival they said, “Take...",river - rivers - water - drowned - vessel - ve...,1.000000,False
2,"They shouted and said, “Where did you cross ov...",-1,-1_mother_she_her_father,"[mother, she, her, father, house, woman, child...","[They said, “ Your love for her is evident, fo...",mother - she - her - father - house - woman - ...,0.000000,False
3,"You did not cross bere, Tell us.” [PER] and hi...",0,0_father_mother_child_birth,"[father, mother, child, birth, pregnant, son, ...","[The news is published, A messenger set out an...",father - mother - child - birth - pregnant - s...,1.000000,False
4,They rejoiced ; the children of [PER] played ;...,4,4_per_pera_pers_perye,"[per, pera, pers, perye, peryesiziba, came, me...","[The birth of [PER]., ‘When [PER] came, the sk...",per - pera - pers - perye - peryesiziba - came...,0.575714,False
...,...,...,...,...,...,...,...,...
3360,In another tale the sea divides at the word of...,-1,-1_mother_she_her_father,"[mother, she, her, father, house, woman, child...","[They said, “ Your love for her is evident, fo...",mother - she - her - father - house - woman - ...,0.000000,False
3361,"And in the Hottentot fables, in like manner, H...",-1,-1_mother_she_her_father,"[mother, she, her, father, house, woman, child...","[They said, “ Your love for her is evident, fo...",mother - she - her - father - house - woman - ...,0.000000,False
3362,‘These facts thow the wide-spread existence of...,-1,-1_mother_she_her_father,"[mother, she, her, father, house, woman, child...","[They said, “ Your love for her is evident, fo...",mother - she - her - father - house - woman - ...,0.000000,False
3363,Dr. Bleck has shown that the Hottentot languag...,41,41_tribes_zulu_africas_tribe,"[tribes, zulu, africas, tribe, cannibals, afri...",[Dr. Bleck has shown that the Hottentot langua...,tribes - zulu - africas - tribe - cannibals - ...,1.000000,True


In [61]:
fig5 = topic_model.visualize_topics()
fig5.write_html("callaway_topics_keybert_orig.html")

In [63]:
fig6 = topic_model.visualize_barchart()
fig6.write_html("callaway_keybert_barchart_orig.html")
fig6

In [66]:
topic_model.reduce_topics(sentences, nr_topics=60)

# Access updated topics
topics = topic_model.topics_

In [67]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1167,-1_mother_she_her_father,"[mother, she, her, father, house, woman, child...","[They said, “ Your love for her is evident, fo..."
1,0,259,0_birth_mother_child_father,"[birth, mother, child, father, children, her, ...","[She gave birth to a child., There is no woman..."
2,1,129,1_cannibal_cannibals_cannibalism_meat,"[cannibal, cannibals, cannibalism, meat, eaten...",[Tame was @ boy whose name was eT the son of U...
3,2,97,2_kwa_ekaya_ikwantandane_tiwa,"[kwa, ekaya, ikwantandane, tiwa, njalo, nga, i...",[So the women swear in like manner : ““ Bane w...
4,3,90,3_heir_heirs_chief_house,"[heir, heirs, chief, house, wives, eldest, inh...","[The house of the chief wife., that is, he is ..."
5,4,81,4_per_pera_pers_perye,"[per, pera, pers, perye, peryesiziba, came, me...","[The birth of [PER]., ‘When [PER] came, the sk..."
6,5,81,5_cattle_cows_cattlekraal_cattleenclosure,"[cattle, cows, cattlekraal, cattleenclosure, s...","[The father said, “Go into the cattle., His fa..."
7,6,73,6_cave_hunted_hunting_tobacco,"[cave, hunted, hunting, tobacco, hunt, trees, ...","[If they caught a man, they went to the cave ;..."
8,7,69,7_river_rivers_water_drowned,"[river, rivers, water, drowned, vessel, vessel...","[the river., On their arrival they said, “Take..."
9,8,60,8_queen_king_kings_queens,"[queen, king, kings, queens, royal, princess, ...","[The old woman said, “The queen has gone away ..."


In [68]:
fig7 = topic_model.visualize_topics()
fig7

In [69]:
fig.write_html("callaway_topics_60.html")

In [70]:
fig = topic_model.visualize_barchart()
fig.write_html("callaway_60_barchart.html")

In [71]:
fig